In [ ]:
import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

In [ ]:
input_ = pd.read_excel('./3.model.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
input_['model'].tolist()

In [ ]:
crawler_status = 'error'

print(crawler_status)

In [ ]:
import requests
import json

In [ ]:
i = 0

jsn = '{"jsn":{"tab":"catalog","make":"' + input_.loc[i, 'Make'] + '","year":"' + input_.loc[i, 'Year'] + '","model":"' + input_.loc[i, 'Model'] + '","nodetype":"model"}}'

payload = {'func': 'navnode_fetch',
           'payload': jsn,
           'api_json_request': '1'}

resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                     data=payload).text

resp = json.loads(resp)

resp

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
html = resp['html_fill_sections']['navchildren[]']

soup = BeautifulSoup(html, 'lxml')

with open('./engine.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

In [ ]:
from lxml import etree

In [ ]:
html = etree.HTML(str(soup))

list_engine = [engine.strip() for engine in html.xpath('//td[@class="nlabel"]/a/text()')]

list_engine

In [ ]:
list_url = ['https://www.rockauto.com'+url.strip() for url in html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')]

list_url

In [ ]:
crawler_status = 'ok'

print(crawler_status)

In [ ]:
if crawler_status == 'ok':
    df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                            'No.': int(input_.loc[i, 'No.']),
                            'Make': input_.loc[i, 'Make'],
                            'Year': input_.loc[i, 'Year'],
                            'Model': input_.loc[i, 'Model'],
                            'Engine': list_engine,
                            'Car_Code': [url.split(',')[-1] for url in list_url],
                            'Url': list_url})
    
    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
else:
    df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                             'No.': int(input_.loc[i, 'No.']),
                             'Make': input_.loc[i, 'Make'],
                             'Year': input_.loc[i, 'Year'],
                             'Model': input_.loc[i, 'Model'],
                             'Url': input_.loc[i, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
    
print(len(output_correct), len(output_error))
output_correct